# Getting routes

After testing out the API in notebook 1, this notebook gets data on travel times to Hattie Mae White on board meeting days. 

In [32]:
import numpy as np
import pandas as pd
import openpyxl
import requests
import time
import dateutil
import datetime

# print all rows and columns when requested 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# working directory

direc = "//Users//afan//Desktop//Misc//HMW_Transit//"

# Hattie Mae White coordinatess

HMW_lat = 29.802759908899148
HMW_long = -95.45410037006431

# API subscription keys

api_key = "4e7902d1a2ae42df978fe20387049854"
api_key2 = "c45886cc797b47319bf15f4d7f84c3e4"

## Bringing in school information 

In [16]:
# cleaned dataset
df_str = direc + "cleaned_data//" + "school_demo_geo.csv"
df_cols = ['School_Num', 'School_Nam', 'X', 'Y', 'End Time']
df = pd.read_csv(endtimes_str, usecols = df_cols, parse_dates=['End Time'])
df.head()

,School_Num,School_Nam,X,Y,End Time
0,'101912001,AUSTIN H S,-95.331465,29.730194,2022-01-11 15:30:00
1,'101912002,BELLAIRE H S,-95.469016,29.691198,2022-01-11 16:10:00
2,'101912003,NORTHSIDE H S,-95.357984,29.782934,2022-01-11 16:10:00
3,'101912004,FURR H S,-95.246987,29.778996,2022-01-11 16:10:00
4,'101912006,JONES FUTURES ACADEMY,-95.339505,29.674387,2022-01-11 15:30:00


In [18]:
# now, let's take a sample school and calculate the route! 
nrow = 2

# school name 
s_name = df.iloc[nrow, 2]

# school end time
s_endtime = (df.iloc[nrow, 4] + timedelta(hours=6)).strftime("%Y-%m-%dT%H:%M:%SZ")

print(s_endtime)

# For some reason, X and Y are longitude and latitude, respectively 
s_lat = df.iloc[nrow, 3]
s_long = df.iloc[nrow, 2]

print(s_lat, s_long)


2022-01-11T20:10:00Z
29.782934091 -95.3579844606


In [19]:
trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&startTime=datetime'{s_endtime}'&$orderby=EndTime&$expand=Legs&subscription-key={api_key}")
   
print(trip.status_code)

200


Now we just gotta do some fun data wrangling 

In [22]:
trip_int = pd.DataFrame(trip.json())
trip_df = trip_int['value'].apply(pd.Series)
trip_df['school'] = df.iloc[nrow, 2]
trip_df.head(1)

ItineraryId                       Created  \
0  f97a4aae-fcf3-43fc-9814-b13baaff528f  2022-01-11T16:50:16.4710683Z   

                                                                                                                            Kml  \
0  http://api.transitiq.com/ODataService.svc/GetItineraryKml?itineraryId=guid'f97a4aae-fcf3-43fc-9814-b13baaff528f'&$format=kml   

              StartTime               EndTime     AdjustedStartTime  \
0  2022-01-11T16:55:39Z  2022-01-11T17:36:45Z  2022-01-11T16:55:39Z   

        AdjustedEndTime            StartStopName                EndStopName  \
0  2022-01-11T17:36:45Z  Fulton North Central NB  W 18Th St @ Northwest Fwy   

        StartStopId         EndStopId StartAddress EndAddress  TransferCount  \
0  Ho414_4620_25037  Ho414_4620_12077         None       None              2   

           TravelTypes  WalkDistance                             RequestId  \
0  Walk,LightTrain,Bus       0.49565  e8e7aa8a-b671-4017-9558-103d286608f1   

    StartLon   StartLat   EndLon    EndLat  LocalTimeOffset  \
0 -95.357984  29.782934 -95.4541  29.80276               -5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [23]:
legs = trip_df['Legs'].apply(pd.Series)
legs.head()

,0,1,2,3,4,5
0,"{'ItineraryLegId': -212348965, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-11T16:50:16.470928Z', 'StopName': '', 'StopId': '', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'DirectionName': '', 'RouteDestinationName': '', 'StartTime': '2022-01-11T16:54:39Z', 'AdjustedStartTime': '2022-01-11T16:54:39Z', 'Duration': 381, 'Length': 0.3284592136973592, 'Address': '', 'ShapeUri': '', 'TransportType': 'Walk', 'Ordinal': 0, 'ItineraryId': 'f97a4aae-fcf3-43fc-9814-b13baaff528f', 'DataSource': None, 'Lat': 29.782934091, 'Lon': -95.3579844606, 'TripId': '', 'TripHeadsign': '', 'DelayFromScheduleInSeconds': None}","{'ItineraryLegId': -212348964, 'AgencyAbbreviation': 'RideMetro', 'AgencyName': 'Houston', 'Created': '2022-01-11T16:50:16.4710165Z', 'StopName': 'Fulton North Central NB', 'StopId': 'Ho414_4620_25037', 'RouteId': 'Ho414_4620_42034', 'RouteName': '700', 'DirectionId': 'Ho414_4620_42034_1', 'DirectionVariantId': 'Ho414_4620_42034_1_1', 'DirectionName': 'METRORail -NORTH LINE TC', 'RouteDestinationName': 'METRORail -NORTH LINE TC', 'StartTime': '2022-01-11T17:02:00Z', 'AdjustedStartTime': '2022-01-11T17:02:00Z', 'Duration': 240, 'Length': 1.2727702054767172, 'Address': '', 'ShapeUri': '', 'TransportType': 'LightTrain', 'Ordinal': 1, 'ItineraryId': 'f97a4aae-fcf3-43fc-9814-b13baaff528f', 'DataSource': None, 'Lat': 29.785456, 'Lon': -95.360757, 'TripId': 'Ho414_4620_9288938', 'TripHeadsign': 'METRORail -NORTH LINE TC', 'DelayFromScheduleInSeconds': 0.0}","{'ItineraryLegId': -212348963, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-11T16:50:16.4710183Z', 'StopName': 'Cavalcade NB', 'StopId': 'Ho414_4620_25041', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'DirectionName': '', 'RouteDestinationName': '', 'StartTime': '2022-01-11T17:12:52Z', 'AdjustedStartTime': '2022-01-11T17:12:52Z', 'Duration': 68, 'Length': 0.0587355160501079, 'Address': '', 'ShapeUri': '', 'TransportType': 'Walk', 'Ordinal': 2, 'ItineraryId': 'f97a4aae-fcf3-43fc-9814-b13baaff528f', 'DataSource': None, 'Lat': 29.802746, 'Lon': -95.368103, 'TripId': '', 'TripHeadsign': '', 'DelayFromScheduleInSeconds': None}","{'ItineraryLegId': -212348962, 'AgencyAbbreviation': 'RideMetro', 'AgencyName': 'Houston', 'Created': '2022-01-11T16:50:16.4710619Z', 'StopName': 'Cavalcade St @ Fulton St', 'StopId': 'Ho414_4620_2235', 'RouteId': 'Ho414_4620_41906', 'RouteName': '026', 'DirectionId': 'Ho414_4620_41906_1', 'DirectionVariantId': 'Ho414_4620_41906_1_0', 'DirectionName': 'MEMORIAL CITY', 'RouteDestinationName': 'MEMORIAL CITY', 'StartTime': '2022-01-11T17:15:00Z', 'AdjustedStartTime': '2022-01-11T17:15:00Z', 'Duration': 1120, 'Length': 5.152803816672346, 'Address': '', 'ShapeUri': '', 'TransportType': 'Bus', 'Ordinal': 3, 'ItineraryId': 'f97a4aae-fcf3-43fc-9814-b13baaff528f', 'DataSource': None, 'Lat': 29.803368, 'Lon': -95.368125, 'TripId': 'Ho414_4620_9262780', 'TripHeadsign': 'MEMORIAL CITY', 'DelayFromScheduleInSeconds': 0.0}","{'ItineraryLegId': -212348961, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-11T16:50:16.4710627Z', 'StopName': 'W 18Th St @ Northwest Fwy', 'StopId': 'Ho414_4620_12077', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'DirectionName': '', 'RouteDestinationName': '', 'StartTime': '2022-01-11T17:34:40Z', 'AdjustedStartTime': '2022-01-11T17:34:40Z', 'Duration': 125, 'Length': 0.10845563780971793, 'Address': '', 'ShapeUri': '', 'TransportType': 'Walk', 'Ordinal': 4, 'ItineraryId': 'f97a4aae-fcf3-43fc-9814-b13baaff528f', 'DataSource': None, 'Lat': 29.80163, 'Lon': -95.454342, 'TripId': '', 'TripHeadsign': '', 'DelayFromScheduleInSeconds': None}","{'ItineraryLegId': -212348960, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-11T16:50:16.4710627Z', 'StopName': '', 'StopId': '', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'Direction

In [24]:
legs.iloc[0].apply(pd.Series)

,ItineraryLegId,AgencyAbbreviation,AgencyName,Created,StopName,StopId,RouteId,RouteName,DirectionId,DirectionVariantId,DirectionName,RouteDestinationName,StartTime,AdjustedStartTime,Duration,Length,Address,ShapeUri,TransportType,Ordinal,ItineraryId,DataSource,Lat,Lon,TripId,TripHeadsign,DelayFromScheduleInSeconds
0,-212348965,None,None,2022-01-11T16:50:16.470928Z,,,,,,,,,2022-01-11T16:54:39Z,2022-01-11T16:54:39Z,381,0.328459,,,Walk,0,f97a4aae-fcf3-43fc-9814-b13baaff528f,None,29.782934,-95.357984,,,NaN
1,-212348964,RideMetro,Houston,2022-01-11T16:50:16.4710165Z,Fulton North Central NB,Ho414_4620_25037,Ho414_4620_42034,700,Ho414_4620_42034_1,Ho414_4620_42034_1_1,METRORail -NORTH LINE TC,METRORail -NORTH LINE TC,2022-01-11T17:02:00Z,2022-01-11T17:02:00Z,240,1.272770,,,LightTrain,1,f97a4aae-fcf3-43fc-9814-b13baaff528f,None,29.785456,-95.360757,Ho414_4620_9288938,METRORail -NORTH LINE TC,0.0
2,-212348963,None,None,2022-01-11T16:50:16.4710183Z,Cavalcade NB,Ho414_4620_25041,,,,,,,2022-01-11T17:12:52Z,2022-01-11T17:12:52Z,68,0.058736,,,Walk,2,f97a4aae-fcf3-43fc-9814-b13baaff528f,None,29.802746,-95.368103,,,NaN
3,-212348962,RideMetro,Houston,2022-01-11T16:50:16.4710619Z,Cavalcade St @ Fulton St,Ho414_4620_2235,Ho414_4620_41906,026,Ho414_4620_41906_1,Ho414_4620_41906_1_0,MEMORIAL CITY,MEMORIAL CITY,2022-01-11T17:15:00Z,2022-01-11T17:15:00Z,1120,5.152804,,,Bus,3,f97a4aae-fcf3-43fc-9814-b13baaff528f,None,29.803368,-95.368125,Ho414_4620_9262780,MEMORIAL CITY,0.0
4,-212348961,None,None,2022-01-11T16:50:16.4710627Z,W 18Th St @ Northwest Fwy,Ho414_4620_12077,,,,,,,2022-01-11T17:34:40Z,2022-01-11T17:34:40Z,125,0.108456,,,Walk,4,f97a4aae-fcf3-43fc-9814-b13baaff528f,None,29.801630,-95.454342,,,NaN
5,-212348960,None,None,2022-01-11T16:50:16.4710627Z,,,,,,,,,2022-01-11T17:34:40Z,2022-01-11T17:34:40Z,125,0.108456,,,Arrived,5,f97a4aae-fcf3-43fc-9814-b13baaff528f,None,29.802760,-95.454100,,,NaN


## Running it on all the schools

Now, let's loop through all the schools and put all the routes together 

In [ ]:
routelist = []

for nrow in range(0, len(df)): 

  s_lat = df.iloc[nrow, 1]
  s_long = df.iloc[nrow, 2]

  # get the trip information for that school
  trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&$orderby=EndTime&$expand=Legs&subscription-key={key}")

  trip1 = pd.DataFrame(trip.json())

  trip_df = trip1['value'].apply(pd.Series)
  trip_df['school'] = df.iloc[nrow, 0]
  trip_df['school_id'] = df.iloc[nrow, 3]

  routelist.append(trip_df)
  print(f"Done with school #{nrow}: {df.iloc[nrow, 0]}")

  time.sleep(2)   

In [ ]:
routes = pd.concat(routelist, ignore_index = True)
routes['AdjustedStartTime'] = routes.AdjustedStartTime.apply(dateutil.parser.isoparse)
routes['AdjustedEndTime'] = routes.AdjustedEndTime.apply(dateutil.parser.isoparse)
routes['length'] = routes['AdjustedEndTime'] - routes['AdjustedStartTime']

all_legs = routes['Legs'].apply(pd.Series)
leglist = []

for route in range(0, len(all_legs)): 

  temp = all_legs.iloc[route, :].apply(pd.Series)
  leglist.append(temp)

legs = pd.concat(leglist)
print(legs.shape)
legs = legs[~np.isnan(legs['Ordinal'])]
legs.shape

In [ ]:
print(f"Number of schools: {len(df)}")
print(f"Number of routes: {len(routes)}")
print(f"Number of legs: {len(legs)}")


In [36]:
(datetime.datetime.combine(datetime.datetime.today(), datetime.time(5, 00)) + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%H:%M:%SZ")

'2022-01-11T11:00:00Z'